In [2]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras import layers

def build_quantized_pilotnet(input_shape=(64, 64, 1), num_outputs=1):
    # Input: float32 (training), simulate uint8 with QuantizeLayer
    inputs = tf.keras.Input(shape=input_shape, dtype=tf.float32, name='input_1')

    # Insert QuantizeLayer to simulate uint8 input
    x = tfmot.quantization.keras.QuantizeLayer()(inputs)

    # Adjusted PilotNet architecture for (64, 64, 1)
    x = layers.Conv2D(24, (5, 5), strides=(2, 2), activation='relu', padding='valid')(x)  # -> (30, 30, 24)
    x = layers.Conv2D(36, (5, 5), strides=(2, 2), activation='relu', padding='valid')(x)  # -> (13, 13, 36)
    x = layers.Conv2D(48, (5, 5), strides=(2, 2), activation='relu', padding='valid')(x)  # -> (5, 5, 48)
    x = layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid')(x)  # -> (3, 3, 64)
    x = layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='valid')(x)  # -> (1, 1, 64)

    x = layers.Flatten()(x)  # -> (64,)

    # Fully connected layers
    x = layers.Dense(100, activation='relu')(x)
    x = layers.Dense(50, activation='relu')(x)
    x = layers.Dense(10, activation='relu')(x)

    # Output layer (no activation, for regression output)
    outputs = layers.Dense(num_outputs, name='output')(x)

    # Create the base model
    base_model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Apply quantization-aware training
    quantized_model = tfmot.quantization.keras.quantize_model(base_model)

    return quantized_model